```
conda install -c conda-forge libxc blas 
pip install --upgrade git+https://gitlab.com/ase/ase.git@master
pip install git+https://gitlab.com/gpaw/gpaw.git
```

On windows, download and install https://visualstudio.microsoft.com/visual-cpp-build-tools/

## Simple properties with single DFT Calculations

### Energy and forces

Two of the most important quantities we are interested in are the total energy and the forces on the atoms.
- Define a calculator and attach it to an `ase.Atoms` object so that `ase` knows how to get the data
- Run DFT calculation

Here is an example of getting the energy and forces from a CO molecule. 



In [ ]:
from ase import Atoms, Atom
from gpaw import GPAW, PW, FermiDirac

# Define molecule
co = Atoms([Atom('C', [0, 0, 0]),
            Atom('O', [1.2, 0, 0])],
           cell=(6., 6., 6.))

# gpaw calculator:
calc = GPAW(mode=PW(350),       # planewave cutoff  350
            xc='PBE',             # the exchange-correlation functional
            nbands=6,             # number of bands
            occupations={'name': 'methfessel-paxton', 'width': 0.01, 'order': 0},    # Methfessel-Paxton smearing
            txt='H.out'    )
co.calc = calc

print('energy = {0} eV'.format(co.get_potential_energy()))
print(co.get_forces())

In [ ]:
from ase import Atoms, Atom
from vasp import Vasp
co = Atoms([Atom('C', [0, 0, 0]),
            Atom('O', [1.2, 0, 0])],
           cell=(6., 6., 6.))
calc = Vasp('molecules/simple-co',  # output dir
            xc='pbe',  # the exchange-correlation functional
            nbands=6,    # number of bands
            encut=350,    # planewave cutoff
            ismear=1,    # Methfessel-Paxton smearing
            sigma=0.01,  # very small smearing factor for a molecule
            atoms=co)
print('energy = {0} eV'.format(co.get_potential_energy()))
print(co.get_forces())

In [1]:
pip install git+https://gitlab.com/gpaw/gpaw.git

     ---------------------------------------- 1.7/1.7 MB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for gpaw
Failed to build gpaw
  Running setup.py install for gpaw: started
  Running setup.py install for gpaw: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [1052 lines of output]
      .git directory not found. GPAW git hash not written.
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-39
      creating build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\ae.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\ah.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\arraydict.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\band_descriptor.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\basis_data.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\berryphase.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\bfield.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\blacs.py -> build\lib.win-amd64-cpython-39\gpaw
      copying gpaw\borncharges.py -> build\lib.win-amd